<a href="https://colab.research.google.com/github/manisggn/Udemy_Deep_learning/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ANN
#Install Tensorflow
!pip install tensorflow
!pip install keras
!pip install matplotlib

#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io




In [0]:
#Importing dataset
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['Churn_Modelling.csv'].decode('utf-8')))






In [0]:
#Defining X and Y
x=df.iloc[:,3:13].values
y=df.iloc[:,13].values

print(x)
type(x)

In [0]:
#enclosing categorical data
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
labelencoder1=LabelEncoder()
x[:,1]=labelencoder1.fit_transform(x[:,1])
labelencoder2=LabelEncoder()
x[:,2]=labelencoder2.fit_transform(x[:,2])

onehotencoder=OneHotEncoder(categorical_features=[1])
x=onehotencoder.fit_transform(x).toarray()
type(x)
x=x[:,1:]
x.shape

In [0]:
#splitting data into training and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [0]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)
x_test.shape
type(x_test)


In [0]:
#Making ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

#Initializing ANN
model=Sequential()

#Adding input layer and first hidden layer
model.add(Dense(units=6, activation='relu', input_dim=11, kernel_initializer='glorot_uniform'))

#Adding 2nd hidden layer
model.add(Dense(units=6, activation='relu',kernel_initializer='glorot_uniform'))

#Adding final layer/output layer
model.add(Dense(units=1, kernel_initializer = 'uniform', activation = 'sigmoid'))

#Compiling ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#Fitting ANN to training set
model.fit(x_train, y_train, batch_size=10, epochs=100)



In [0]:
#Predicting test set results
y_pred= model.predict(x_test)
y_pred= (y_pred>0.5)


In [0]:
#Making confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

In [0]:
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""

new_pred= model.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_pred=(new_pred>0.5)
new_pred

In [0]:
#Evaluation of ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def for_validation():
  model= Sequential()
  model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
  model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
  model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

model = KerasClassifier(build_fn = for_validation, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv = 10)
mean = accuracies.mean()
variance = accuracies.std()


In [0]:
#Tuning ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def for_validation(optimizer):
  model = Sequential()
  model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
  model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
  model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

model=KerasClassifier(build_fn= for_validation)
parameters= {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

grid_search= GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(x_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
  
